# FlexTag Parser Testing
This notebook tests the core parser functionality step by step

In [6]:
# Setup
from pathlib import Path
from ..managers.base import BaseManager
from ..managers.data import DataManager
from ..managers.parser import ParserProvider


# Initialize managers
# data_manager = DataManager()
# parser_provider = ParserProvider()
# data_manager.register("parsers", "default", parser_provider)
# 
# print("Setup complete")

ModuleNotFoundError: No module named 'base'

In [ ]:
# Test 1: Simple block parsing
test_data = """
[[DATA:block1 #tag1 .path.to.item]]
This is content
[[/DATA]]
"""

# Parse and collect blocks
blocks = list(parser_provider.parse_string(test_data))

print("Expected: 1 block with id='block1', tags=['tag1'], paths=['path.to.item']")
print(f"\nGot: {len(blocks)} block(s)")
print(f"ID: {blocks[0].id}")
print(f"Tags: {blocks[0].tags}")
print(f"Paths: {blocks[0].paths}")
print(f"Content lines: {blocks[0].content_start} to {blocks[0].content_end}")

In [ ]:
# Test 2: Block with JSON parameters
test_data = """
[[DATA:block2 #config {"format": "json", "version": 1.0}]]
{
    "key": "value"
}
[[/DATA]]
"""

blocks = list(parser_provider.parse_string(test_data))

print("Expected: 1 block with parameters containing format='json' and version=1.0")
print(f"\nGot: Parameters = {blocks[0].parameters}")

In [ ]:
# Test 3: Multiple blocks
test_data = """
[[DATA:first #tag1]]
Content 1
[[/DATA]]

[[DATA:second #tag2]/]]

[[DATA:third #tag3 {"note": "test"}]]
Content 3
[[/DATA]]
"""

blocks = list(parser_provider.parse_string(test_data))

print(f"Expected: 3 blocks (one self-closing)")
print(f"\nGot: {len(blocks)} blocks")
for block in blocks:
    print(f"\nBlock ID: {block.id}")
    print(f"Is self-closing: {block.content_start == block.content_end}")

In [ ]:
# Test 4: Error handling
test_data = """
[[DATA:error #tag {invalid_json}]]
Content
[[/DATA]]
"""

print("Expected: JSON parsing error")
try:
    blocks = list(parser_provider.parse_string(test_data))
    print("Error: No exception raised")
except json.JSONDecodeError as e:
    print(f"Got expected error: {str(e)}")

In [ ]:
# Test 5: File handling
# Create a temp test file
test_file = Path("test.flextag")
test_file.write_text("""
[[DATA:file1 #test]]
File content
[[/DATA]]
""")

blocks = list(parser_provider.parse_file(test_file))

print("Expected: 1 block with file_path set")
print(f"\nGot: file_path = {blocks[0].file_path}")

# Cleanup
test_file.unlink()